In [ ]:
%pwd

In [ ]:
%cd Deeplifting/

In [ ]:
%ls

In [ ]:
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
# stdlib
import click
import os
import warnings
from datetime import datetime
from itertools import product

# third party
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
from pygranso.private.getNvar import getNvarTorch
from pygranso.pygranso import pygranso
from pygranso.pygransoStruct import pygransoStruct

# first party
from config import (
    low_dimensional_problem_names,
    search_agg_functions,
    search_hidden_activations,
    search_hidden_sizes,
    search_include_bn,
    search_input_sizes,
    search_output_activations,
)
from deeplifting.models import ReLUDeepliftingMLP
from deeplifting.optimization import deeplifting_predictions, deeplifting_nd_fn
from deeplifting.problems import HIGH_DIMENSIONAL_PROBLEMS_BY_NAME, PROBLEMS_BY_NAME
from deeplifting.utils import get_devices, initialize_vector, set_seed, train_model_to_output

# Filter warnings
warnings.filterwarnings('ignore')

In [ ]:
torch.cuda.is_available()

In [ ]:
!nvidia-smi

In [ ]:
from tasks import find_best_architecture_task

In [ ]:
problem_name = 'ackley'
method = 'deeplifting-pygranso'
dimensionality = 'low-dimensional'
experimentation = True

In [ ]:
find_best_architecture_task(
    problem_name=problem_name,
    method=method,
    dimensionality=dimensionality,
    experimentation=False,
    include_weight_initialization=True,
)

In [ ]:
%debug

# Create Distance Histogram

In [ ]:
problem_name = 'ackley'
problem = PROBLEMS_BY_NAME[problem_name]

bounds = problem['bounds']
units = 128
input_dimension = 1

device = torch.device('cpu')

# Distances
distances = []
other_distances = []

# Starting points
dl_init_values = []
init_values = []

for trial in range(1000):
    # Inputs
    inputs = torch.randn(size=(input_dimension, units))
    inputs = inputs.to(device=device, dtype=torch.double)

    # Creates different weight intializations for the same starting point
    # x0
    # Deeplifting model with skip connections
    model = ReLUDeepliftingMLP(
        initial_hidden_size=units,
        hidden_sizes=(units,) * 10,
        output_size=2,
        bounds=bounds,
        initial_layer_type='linear',
        include_weight_initialization=True,
        include_bn=True,
        seed=trial,
    )

    model = model.to(device=device, dtype=torch.double)

    # Print the model outputs and check against x0 also
    # want to use a print out to make sure all models have
    # the same starting point
    model.eval()
    outputs = model(inputs=inputs)
    dl_init_values.append(outputs.detach().numpy())
    
    # Save the distances
    distances.append(np.linalg.norm(outputs.detach().numpy(), 2))
    
    x_start = initialize_vector(size=2, bounds=bounds)
    init_values.append(x_start)
    other_distances.append(np.linalg.norm(x_start, 2))

In [ ]:
ax = pd.Series(distances).hist(color='dodgerblue', alpha=0.6, bins=20)
pd.Series(other_distances).hist(color='orange', alpha=0.6, bins=20)

In [ ]:
pd.Series(distances).describe()

In [ ]:
pd.Series(other_distances).describe()

In [ ]:
ax = (
    pd.DataFrame(dl_init_values, columns=['x1', 'x2'])
    .plot
    .scatter(x='x1', y='x2')
)

(
    pd.DataFrame(init_values, columns=['x1', 'x2'])
    .plot
    .scatter(x='x1', y='x2', color='orange', ax=ax)
)

In [ ]:
X_deeplifting = pd.DataFrame(distances, columns=['distance'])
X_deeplifting['init_type'] = 'Neural Network'

X_init = pd.DataFrame(other_distances, columns=['distance'])
X_init['init_type'] = 'Uniform Initialization'

X = pd.concat([X_deeplifting, X_init])

In [ ]:
X['binned_distances'] = pd.cut(X['distance'], bins=range(0, 55, 5))
X

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

ax = sns.countplot(data=X, x='binned_distances', hue='init_type')
ax.set_title('$x^0$ to $x^{g}$ Distance')
ax.set_xlabel('Distance Bins')
ax.set_ylabel('Frequency')
ax.grid()
plt.xticks(rotation=90)

In [ ]:
ax = (
    pd.DataFrame([np.sin(i) for i in dl_init_values], columns=['x1', 'x2'])
    .plot
    .scatter(x='x1', y='x2')
)

In [ ]:
inputs = torch.randn(64, 1000)

In [ ]:
import torch.nn as nn

nn.Linear(1000, 64)(inputs)

In [ ]:
x = torch.tensor([1, 2, 3, 4])

In [ ]:
type(model)

In [ ]:
type(x)

In [ ]:
objective = problem['objective']

In [ ]:
fn = lambda x: objective(x)

In [ ]:
from typing import Callable

In [ ]:
type(torch.device('cpu'))